## Mask Detection Implementation ##

Detect if a person on the web camera triggered using OpenCv is wearing mask or not using the Mask_NoMask.h5 model trained using tensorflow.




In [ ]:
## Import tensorflow lib to load the model 
import tensorflow as tf

In [26]:
# conda install conda-forge::opencv

Channels:
 - defaults
 - conda-forge
Platform: osx-arm64
Solving environment: / warning  libmamba Added empty dependency for problem type SOLVER_RULE_UPDATE
done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - conda-forge::opencv


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    aom-3.9.1                  |       h7bae524_0         2.1 MB  conda-forge
    arrow-cpp-13.0.0           |  hce30654_49_cpu          42 KB  conda-forge
    aws-c-auth-0.7.22          |      h8a62e84_10          89 KB  conda-forge
    aws-c-cal-0.7.1            |       h94d0942_1          39 KB  conda-forge
    aws-c-common-0.9.23        |       h99b78c6_0         215 KB  conda-forge
    aws-c-compression-0.2.18   |       h94d0942_7          18 KB  conda-forge
    aws-c-event-stream-0.4.2   |      hb74cd8f_15          46 KB  conda-forge
    aws-c-http-0.8.2           |  

In [27]:
## Import important libraries for prediction
from keras.utils import load_img, img_to_array
from keras.models import Model, load_model
import cv2

In [28]:
import numpy as np

In [29]:
## Loading the pre-trained mask detection model
model = load_model("Mask_NoMask.h5")

/opt/anaconda3/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [ ]:
### Starting the camera
cam = cv2.VideoCapture(0)

###Detect Face
face_class = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    _, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_class.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        face = img[y:y+h, x:x+w]
        face = cv2.resize(face, (224,224))
        face = img_to_array(face)
        face = np.expand_dims(face, axis = 0)
        face = preprocess_input(face)
        
        text = ""
        colour = (0,0,0)
        (mask, withoutmask) = model.predict(face)[0]
        if mask > withoutmask:
            text = 'Mask Detected'
            colour = (0,225,0)
        else:
            text = 'No Mask'
            colour = (0,0,225)
        
        
        cv2.rectangle(img, (x, y), (x+w, y+h), colour, 2)
        cv2.putText(img , text, (x, y -15), cv2.FONT_HERSHEY_SIMPLEX, 0.9, colour, 2)
    
    cv2.imshow('My Mask Detector', img)
    if cv2.waitKey(1) == 13:
        break
        
cam.release()
cv2.destroyAllWindows()